# Sample-based quantum diagonalization of a chemistry Hamiltonian
https://quantum.cloud.ibm.com/docs/en/tutorials/sample-based-quantum-diagonalization

In [6]:
import pyscf
import pyscf.cc
import pyscf.mcscf
import ffsim
import numpy as np
import matplotlib.pyplot as plt

import qiskit
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

import qiskit_ibm_runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler

import qiskit_addon_sqd

print('pyscf version:', pyscf.__version__)
print('Qiskit version: ', qiskit.__version__)
print('Qiskit Runtime version: ', qiskit_ibm_runtime.__version__)

import importlib.metadata
print('qiskit_addon_sqd version: ', importlib.metadata.version('qiskit_addon_sqd'))
print('ffsim:', importlib.metadata.version('ffsim'))

pyscf version: 2.10.0
Qiskit version:  2.2.1
Qiskit Runtime version:  0.42.0
qiskit_addon_sqd version:  0.12.0
ffsim: 0.0.60


# 고전 화학 계산

In [7]:
open_shell = False # Closed Shell: 최종 솔루션에서 RHF 그라운드 스테이트가 가장 큰 비중을 차지함
spin_sq = 0 # 총 스핀 제곱(S^2)이 0인 Singlet state

# gto: Gaussian type orbitals
# Mole() : 분자 객체
mol = pyscf.gto.Mole() 

# N2 분자 생성
mol.build(
    # 질소 원자의 위치 x축 1 옹스트롬
    atom=[["N", (0, 0, 0)], ["N", (1.0, 0, 0)]], 
    
    # cc-pvdz: 기저함수, sto-3g보다 정확도 높은 표준 기저 함수 중 하나
    basis="cc-pvdz", 
    
    # D-infinitiy-h 대칭성 :계산을 효율적으로 하게 함
    # D: 주축(x축) 
    # infinite: 축을따라 임의의 각도로 돌려도 대칭임
    # h: horizontal, N끼리 거울면 대칭
    symmetry="Dooh", 
)

# Active Space 정의
# 가장 에너지 낮은 2개의 sptial orbital 동결 (4개의 큐빗 아낌)
# 공간 오비탈 28 -> 26 (큐빗 56 -> 52)
n_frozen = 2
active_space = range(n_frozen, mol.nao_nr()) # mol.nao_nr(): 전체 공간 오비탈의 수
 
# 생성된 mol 기반으로 RHF(restrict Hartree-Fock) 계산: 분자 오비탈 계산
# scf.mo_coeff: 계산된 분자 오비탈 (MO)이 어떤 원자 오비탈(AO)의 조합으로 이루어져 있는지 나타내는 계수 행렬
# scf.mo_energy: 각 분자 오비탈의 에너지
# scf.mo_occ: 각 분자 오비탈에 전자가 몇 개 채워졌는지 (0.0 또는 2.0) 나타냄 (낮은 에너지부터 채워짐)
# Output: RHF 에너지(하나의 Slater Determinant로 근사), converged SCF energy = ...
scf = pyscf.scf.RHF(mol).run() 

num_orbitals = len(active_space) # 26: 낮은 에너지부터 0번
n_electrons = int(sum(scf.mo_occ[active_space])) # 총 전자 개수 = 10

# Open Shell 이면 계산 달라짐
num_elec_a = (n_electrons + mol.spin) // 2 # 업스핀 수
num_elec_b = (n_electrons - mol.spin) // 2 # 다운스핀 수

# MCSCF(Multi-Configurational Self-Consistent Field) (RHF, 또는 SCF) 방법보다 더 정확하고 강력한 상위 레벨의 양자 화학 계산법
# CASCI (Complete Active Space CI): MCSCF의 한 종류
# CASCI (Complete Active Space Configuration Interaction, 완전 활성 공간 배치 상호작용) 기능을 사용하기 위한 객체(cas) 생성
cas = pyscf.mcscf.CASCI(scf, num_orbitals, (num_elec_a, num_elec_b))

mo = cas.sort_mo(active_space, base=0) # 활성 공간 오비탈 골라내고 정렬, 활성오비탈 0번부터 시작

# 1전자 적분 h_pq 계산
# nuclear_repulsion_energy: 핵과 핵 사이의 반발 에너지입, 분자가 고정되어 있으므로 이 값은 상수
hcore, nuclear_repulsion_energy = cas.get_h1cas(mo)

# cas.get_h2cas(mo): 2전자 적분 (pq|rs) 계산 -> 1차원 배열로 반환
# pyscf.ao2mo.restore: 1차원 배열을 4차원 텐서(26 26 26 26)로 복원
# 1: 4차원 텐서로 복원하라는 옵션
# num_orbitals: 복원할 텐서의 크기
eri = pyscf.ao2mo.restore(1, cas.get_h2cas(mo), num_orbitals) 

# 26개 오비탈 문제를 고전 컴퓨터에서 정밀한 방식(SCI)으로 미리 계산해 둔 벤치마크 값
exact_energy = -109.22690201485733

converged SCF energy = -108.929838385609


In [8]:
# LUCJ 회로에 사용할 파라미터를 고전 컴퓨터로 계산
# pyscf.cc.CCSD(...): PySCF의 CCSD(coupled cluster singles and doubles) 계산 준비
# scf: reference state로 rhf 계산 결과를 참조
# frozen: 동결시킬 오비탈
# Output: CCSD 방법으로 근사한 그라운드 스테이트 에너지와 E_ccsd - E_RHF 값
ccsd = pyscf.cc.CCSD(
    scf, frozen=[i for i in range(mol.nao_nr()) if i not in active_space]
).run()

# t1, t2가 K, J 계수들을 만들기 위한 재료
t1 = ccsd.t1
t2 = ccsd.t2

E(CCSD) = -109.2177884185544  E_corr = -0.2879500329450037


# 양자 샘플링

In [ ]:
# ffsim과 qiskit을 이용한 양자 회로 설계

n_reps = 1 # 회로 레이어

# 텀들중에서 ibm의 heavy-hex 구조에 맞는 상호작용만 남김
# Gemini:
 # (p, p+1), (p, p) 도 서로 에너지가 가까운 오비탈이지만, 그렇다고 계산에 반드시 중요한 텀이라는것은 아님 
 # 이 또한 ibm topology에 맞추기 위한것
 # 중요성을 t2 진폭의 크기를 기준으로 판단하는게 나을것
alpha_alpha_indices = [(p, p + 1) for p in range(num_orbitals - 1)] # 같은 스핀끼리 상호작용 쌍 정의
alpha_beta_indices = [(p, p) for p in range(0, num_orbitals, 4)] # 다른 스핀끼리 상호작용 쌍 정의
 
# ucj_op: UCJ 회로를 나타내는 operator 생성
# UCJOpSpinBalanced: 스핀 밸런스드 -> J나 K에 스핀 인덱스가 없음, 업스핀 다운스핀이 같음
# from_t_amplitudes: 이 연산자들을 t 진폭으로부터 만든다
ucj_op = ffsim.UCJOpSpinBalanced.from_t_amplitudes(
    t2=t2,
    t1=t1,
    n_reps=n_reps,
    interaction_pairs=(alpha_alpha_indices, alpha_beta_indices),
    # t2 텐서를 최적화(압축)하고, 압축된 t2를 통해 K와 J를 계산함
    optimize=True,
    # t2 텐서 최적화를 1000번 안쪽으로만 함
    options=dict(maxiter=1000),
)
 
nelec = (num_elec_a, num_elec_b)

# create an empty quantum circuit
qubits = QuantumRegister(2 * num_orbitals, name="q")
circuit = QuantumCircuit(qubits)

# 하트리폭 상태를 만듬 |000...0111...1 000...0111...1>
circuit.append(ffsim.qiskit.PrepareHartreeFockJW(num_orbitals, nelec), qubits)
# apply the UCJ operator to the reference state
circuit.append(ffsim.qiskit.UCJOpSpinBalancedJW(ucj_op), qubits)
circuit.measure_all()
# display(circuit.draw('mpl'))

In [5]:
QiskitRuntimeService.save_account(
    token="API",
    instance="CRN",
    overwrite=True,
    set_as_default=True
)

service = QiskitRuntimeService()
print(service.backends())
backend_name = "ibm_torino" 
backend = service.backend(backend_name)
print(backend)

management.get:WARNING:2025-10-21 15:44:51,539: Loading default saved account


[<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_torino')>]
<IBMBackend('ibm_torino')>


In [30]:
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
isa_circ = pm.run(circuit)
sampler = Sampler(mode=backend)
#job = sampler.run([isa_circ], shots=100000)
print(f">>> Job ID: {job.job_id()}")

>>> Job ID: d3q93jbgrqts73854i20


In [28]:
job = service.job('d3q93jbgrqts73854i20')
result = job.result()
print(dir(result))
#print(dir(result[0].data))
count = result[0].data.meas
#print(type(count))
count

['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_is_protocol', '_metadata', '_pub_results', 'metadata']


BitArray(<shape=(), num_shots=100000, num_bits=52>)

# Configuration Recovery 및 결과

In [29]:
from functools import partial

from qiskit_addon_sqd.fermion import (
    SCIResult,
    diagonalize_fermionic_hamiltonian,
    solve_sci_batch,
)

# SQD 이터레이션 중지 조건
energy_tol = 1e-3 # 에너지 변화
occupancies_tol = 1e-3 # 오비탈 점유율 n, ( n_(i+1)-n_(i) )의 infinite norm으로 정의
max_iterations = 5 # 최대 5번

# Batch와 대각화 관련 옵션
num_batches = 3 # Batch의 수 (K)
samples_per_batch = 300 # Batch의 크기
symmetrize_spin = True # 스핀 대칭성 고려 결과 개선
# configuration recovery를 위한 샘플링때 계수가 큰 bitstring들을 다음 이터레이션을 위한 샘플링에 추가
carryover_threshold = 1e-4 
max_cycle = 200 # Davidson 알고리즘등의 반복을 200번으로 제한

# spin_sq = 0.0: 스핀 symmetry 관련 패널티항 추가 
# H + lambda(S^2 + s(s+1))^2 여기서 s(s+1)=0 을 넣는다는 뜻
sci_solver = partial(solve_sci_batch, spin_sq=0.0, max_cycle=max_cycle)
result_history = []

# 결과 출력용 함수
def callback(results: list[SCIResult]):
    result_history.append(results)
    iteration = len(result_history)
    print(f"Iteration {iteration}")
    for i, result in enumerate(results):
        print(f"\tSubsample {i}")
        print(f"\t\tEnergy: {result.energy + nuclear_repulsion_energy}")
        print(
            f"\t\tSubspace dimension: {np.prod(result.sci_state.amplitudes.shape)}"
        )

# initial n 설정 (노이즈 심해서 유효한 샘플이 없을때 RHF 상태를 기준으로 함)
initial_occupancies_spin_orbital = np.zeros(2 * num_orbitals)
initial_occupancies_spin_orbital[:num_elec_a] = 1.0
initial_occupancies_spin_orbital[num_orbitals : num_orbitals + num_elec_b] = 1.0


result = diagonalize_fermionic_hamiltonian(
    hcore,                     # 1-전자 적분 (활성 공간)
    eri,                       # 2-전자 적분 (활성 공간)
    count,                     # 양자 측정 결과 (BitArray 객체)
    samples_per_batch=samples_per_batch, # 배치당 샘플 수 (300)
    norb=num_orbitals,         # 활성 공간 오비탈 수 (26)
    nelec=nelec,               # 활성 공간 전자 수 ((5, 5))
    num_batches=num_batches,   # 배치 수 (3)
    energy_tol=energy_tol,     # 에너지 수렴 기준 (0.001)
    occupancies_tol=occupancies_tol, # n 수렴 기준 (0.001), l maximum norm
    max_iterations=max_iterations, # 최대 반복 횟수 (5)
    sci_solver=sci_solver,     # 계산 함수 위에서 partial 통해서 정의
    symmetrize_spin=symmetrize_spin, # 스핀 inversion symmetry 고려 방법 사용 (sqrt(d)/2개 뽑아서)
    carryover_threshold=carryover_threshold, # 중요 배치 유지 기준 (1e-4)
    callback=callback,         # 반복 시 호출될 함수
    seed=12345,                # 재현성을 위한 랜덤 시드
    initial_occupancies=initial_occupancies_spin_orbital, # initial n 설정
)

Iteration 1
	Subsample 0
		Energy: -101.42152153469274
		Subspace dimension: 299209
	Subsample 1
		Energy: -103.79573138771669
		Subspace dimension: 285156
	Subsample 2
		Energy: -103.63968025606016
		Subspace dimension: 311364
Iteration 2
	Subsample 0
		Energy: -106.83788453190584
		Subspace dimension: 301401
	Subsample 1
		Energy: -108.0158350981591
		Subspace dimension: 300304
	Subsample 2
		Energy: -108.01026022311552
		Subspace dimension: 299209
Iteration 3
	Subsample 0
		Energy: -108.17500045702255
		Subspace dimension: 356409
	Subsample 1
		Energy: -108.09882296699314
		Subspace dimension: 362404
	Subsample 2
		Energy: -108.14950934775523
		Subspace dimension: 354025
Iteration 4
	Subsample 0
		Energy: -109.05469081305813
		Subspace dimension: 446224
	Subsample 1
		Energy: -109.04225501224518
		Subspace dimension: 421201
	Subsample 2
		Energy: -109.03905158681414
		Subspace dimension: 401956
Iteration 5
	Subsample 0
		Energy: -109.09915172595288
		Subspace dimension: 591361
	Subs